In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from scipy.io import loadmat
from PIL import Image
import numpy as np


mnist = loadmat(r"C:\Users\sivar\Downloads\mnist-original.mat\mnist-original.mat")
images = mnist["data"].T
labels = mnist["label"][0]

# Defining data transformations for preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the pixel values
])


# Converting data into PyTorch Dataset
dataset = torch.utils.data.TensorDataset(torch.Tensor(images), torch.Tensor(labels))

# Create DataLoader for batching and shuffling
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split



# Converting data into PyTorch Dataset
dataset = torch.utils.data.TensorDataset(torch.Tensor(images), torch.Tensor(labels))

# Defining the ratio for splitting 
train_ratio = 0.8
test_ratio = 0.2

# Spliting dataset into train and test sets using sklearn
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=test_ratio, random_state=42)

# Defining samplers for train and test sets
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Creating DataLoader for both train and test sets
batch_size = 32

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)



In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Defining the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Setting up the device (GPU if available, else CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Initializing the model, loss function, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the CNN
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Epoch [{epoch + 1}/{epochs}], "
                  f"Batch [{i + 1}/{len(train_loader)}], "
                  f"Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

print("Finished Training")


100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:03<00:00, 2804127.18it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 2539372.66it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 994119.05it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 2203901.99it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch [1/5], Batch [100/938], Loss: 0.7551
Epoch [1/5], Batch [200/938], Loss: 0.1806
Epoch [1/5], Batch [300/938], Loss: 0.1215
Epoch [1/5], Batch [400/938], Loss: 0.1101
Epoch [1/5], Batch [500/938], Loss: 0.0876
Epoch [1/5], Batch [600/938], Loss: 0.0821
Epoch [1/5], Batch [700/938], Loss: 0.0616
Epoch [1/5], Batch [800/938], Loss: 0.0639
Epoch [1/5], Batch [900/938], Loss: 0.0556
Epoch [2/5], Batch [100/938], Loss: 0.0518
Epoch [2/5], Batch [200/938], Loss: 0.0637
Epoch [2/5], Batch [300/938], Loss: 0.0463
Epoch [2/5], Batch [400/938], Loss: 0.0413
Epoch [2/5], Batch [500/938], Loss: 0.0493
Epoch [2/5], Batch [600/938], Loss: 0.0366
Epoch [2/5], Batch [700/938], Loss: 0.0482
Epoch [2/5], Batch [800/938], Loss: 0.0410
Epoch [2/5], Batch [900/938], Loss: 0.0339
Epoch [3/5], Batch [100/938], Loss: 0.0265
Epoch [3/5], Batch [200/938], Loss: 0.0317
Epoch [3/5], Batch [300/938], Loss: 0.0257
Epoch [3/5], Batch [40

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

model.eval()  # Setting the model to evaluation mode

predicted_labels = []
true_labels = []

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculating accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Calculating confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 98.81%
Confusion Matrix:
[[ 979    0    0    0    0    0    0    1    0    0]
 [   0 1134    0    0    0    1    0    0    0    0]
 [   2    2 1019    1    4    0    0    2    2    0]
 [   1    0    0  995    0    8    0    0    5    1]
 [   0    0    0    0  980    0    0    0    0    2]
 [   2    0    0    3    0  882    3    0    0    2]
 [  11    2    0    0    9    0  935    0    1    0]
 [   1    7    3    1    0    0    0 1011    1    4]
 [   2    1    0    1    3    1    1    0  960    5]
 [   5    0    0    1   12    1    0    4    0  986]]
